In [ ]:
import json
import requests
from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph

# Qoob API 설정 (실제 키로 교체 필요)
QOOB_API_KEY = "your_qoob_api_key"
QOOB_API_URL = "https://api.qoob.ai/v1/chat/completions"

# 상태 정의
class CafeState(TypedDict):
    messages: List[Dict[str, str]]
    current_menu: str
    menu_db: Dict[str, Dict]
    should_continue: bool

# 메뉴 DB 로드 함수
def load_menu_db(filepath: str) -> Dict[str, Dict]:
    with open(filepath, "r", encoding="utf-8") as f:
        return json.load(f)

# Qoob API 호출 함수
def call_qoob_api(prompt: str, context: str = "") -> str:
    headers = {
        "Authorization": f"Bearer {QOOB_API_KEY}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "qoob-4",
        "messages": [
            {"role": "system", "content": "당신은 카페 메뉴 전문가입니다. 친절하고 전문적으로 답변해주세요."},
            {"role": "user", "content": f"{context}\n\n{prompt}"}
        ],
        "temperature": 0.7
    }
    
    try:
        response = requests.post(QOOB_API_URL, headers=headers, json=data, timeout=10)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        return f"⚠️ 오류 발생: {str(e)}"

# 메뉴 상세 정보 출력 함수
def display_full_menu_info(menu: str, db: Dict[str, Dict]) -> str:
    info = db.get(menu, {})
    if not info:
        return "해당 메뉴 정보를 찾을 수 없습니다."
    
    # 추천 디저트 생성 (실제 호출은 호출하는 곳에서 함)
    dessert_pairing = call_qoob_api(
        f"다음 커피 메뉴와 잘 어울리는 디저트를 2가지 추천해주고 간단히 설명해주세요: {menu}",
        f"메뉴 정보: {info}"
    )
    
    return f"""
==============================
🍵 {menu} 상세 정보
==============================
• 설명: {info.get('description', '정보 없음')}
• 가격: {info.get('price', 0):,}원
• 주요 재료: {', '.join(info.get('ingredients', []))}
• 추천 디저트: {dessert_pairing}
==============================
"""

def init_state(state: CafeState) -> dict:
    if not state:
        state = {
            "messages": [{"role": "system", "content": "카페 메뉴 안내 시스템을 시작합니다."}],
            "current_menu": "",
            "menu_db": load_menu_db("C:/mylangchain/langchain_basic/data/cafe_menu_data.txt"),
            "should_continue": True
        }
    return state

def greeting(state: CafeState) -> dict:
    greeting_msg = call_qoob_api(
        "카페 메뉴 도우미로서 친절하게 인사하고, 메뉴 조회 방법을 안내해주세요.",
        "현재 제공 메뉴: " + ", ".join(state["menu_db"].keys())
    )
    # append 제거: .ipynb에서 메시지가 누적되는 문제 방지
    print(f"\n{greeting_msg}")
    return state

def get_user_input(state: CafeState) -> dict:
    user_input = input("\n👤 사용자: ").strip()
    
    if user_input.lower() in ["종료", "exit", "quit", "y"]:
        state["should_continue"] = False
        return state
    
    if not user_input:
        # append 제거
        print("메뉴 이름을 입력해주세요. 예) '카푸치노'")
        return state
    
    # 메뉴 인식 로직
    menu = ""
    for menu_name in state["menu_db"].keys():
        if menu_name.lower() in user_input.lower():
            menu = menu_name
            break
    
    state["current_menu"] = menu if menu else ""
    # append 제거
    return state

def check_continue(state: CafeState) -> str:
    return "end" if not state["should_continue"] else "provide_info"

def provide_menu_info(state: CafeState) -> dict:
    menu = state["current_menu"]
    db = state["menu_db"]
    
    if menu:
        full_info = display_full_menu_info(menu, db)
        print(full_info)
        
        # 추가 꿀팁 제공
        tips = call_qoob_api(
            f"이 메뉴를 더 맛있게 즐기는 방법을 알려주세요: {menu}",
            f"메뉴 정보: {db.get(menu, {})}"
        )
        print(f"\n💡 팁: {tips}\n")
        
        # append 제거
    else:
        available_menus = "\n- " + "\n- ".join(db.keys())
        response = f"해당 메뉴를 찾을 수 없습니다. 현재 제공 메뉴:{available_menus}\n원하시는 메뉴 이름을 정확히 입력해주세요."
        print(response)
        # append 제거
    
    return state

def end_conversation(state: CafeState) -> dict:
    farewell = call_qoob_api("작별 인사를 친절하게 해주세요. 고객이 다시 방문하고 싶은 마음이 들도록 해주세요.")
    print(f"\n{farewell}")
    # append 제거
    return state

# 그래프 구성 --------------------------------------------------
workflow = StateGraph(CafeState)

# 노드 추가
workflow.add_node("init", init_state)
workflow.add_node("greet", greeting)
workflow.add_node("get_input", get_user_input)
workflow.add_node("check_continue", check_continue)
workflow.add_node("provide_info", provide_menu_info)
workflow.add_node("end", end_conversation)

# 흐름 구성
workflow.set_entry_point("init")
workflow.add_edge("init", "greet")
workflow.add_edge("greet", "get_input")
workflow.add_edge("get_input", "check_continue")

# 계속 여부 확인 분기
workflow.add_conditional_edges(
    "check_continue",
    lambda state: "end" if not state["should_continue"] else "provide_info",
)

# 정보 제공 후 다시 시작
workflow.add_edge("provide_info", "greet")
workflow.set_finish_point("end")

# 컴파일
cafe_bot = workflow.compile()

# 실행 --------------------------------------------------
if __name__ == "__main__":
    print("\n" + "="*50)
    print("🍵 프리미엄 카페 메뉴 안내 시스템 (Qoob AI 활용) 🍰")
    print("="*50)
    
    # 초기 상태
    state = {}
    
    # 실행 예시
    print("\n실행 예시:")
    print("👤 사용자: 카푸치노")
    print("""
==============================
🍵 카푸치노 상세 정보
==============================
• 설명: 에스프레소, 스팀 밀크, 우유 거품이 1:1:1 비율...
• 가격: 5,000원
• 주요 재료: 에스프레소, 스팀 밀크, 우유 거품
• 추천 디저트: 1. 티라미수 - 커피와 잘 어울리는 ... 
              2. 크로와상 - 바삭함과 커피의 조화...
==============================
""")
    print("👤 사용자: 종료")
    print("감사합니다! 또 방문해주세요 :)")
    print("="*50)
    
    # 시스템 실행
    state = cafe_bot.invoke(state)
    while state.get("should_continue", True):
        state = cafe_bot.invoke(state)
